In [1]:
import iris
import numpy as np
import itertools
import datetime
import glob
import os
from bokeh.plotting import figure, show, save
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Patches, Plot

In [2]:
forecast_date_time = datetime.datetime(2020, 11, 1, 12)
str_year, str_month, str_day, str_hour = str(forecast_date_time.year), \
                                             str('%02d' % forecast_date_time.month), \
                                             str('%02d' % forecast_date_time.day), \
                                             str('%02d' % forecast_date_time.hour)

date_label = '%s%s%s_%sZ' % (str_year, str_month, str_day, str_hour)
out_data_dir = os.path.join('/scratch/hadpx/cold_surge_monitoring/mogreps/processed_data', str_year, str_month, str_day, str_hour)
u850_file_name = os.path.join(out_data_dir, 'MOG_x_wind_850_24H_%s.nc' % (date_label))
u850_cube = iris.load_cube(u850_file_name)

v850_file_name = os.path.join(out_data_dir, 'MOG_y_wind_850_24H_%s.nc' % (date_label))
v850_cube = iris.load_cube(v850_file_name)
speed_cube = (u850_cube**2 + v850_cube**2)**0.5
u850_ens_mean = u850_cube.collapsed('realization', iris.analysis.MEAN)
v850_ens_mean = v850_cube.collapsed('realization', iris.analysis.MEAN)
speed_ens_mean = speed_cube.collapsed('realization', iris.analysis.MEAN)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/

In [3]:
import math
import numpy as np
from bokeh.palettes import Viridis11
class vector():
    '''
    Generates vectors for plotting winds.
    '''    
    def __init__(self, u, v, **kwargs):
        '''
        '''
        # Get arguments
        xSkip = kwargs.get("xSkip", 2)
        ySkip = kwargs.get("ySkip", 2)
        maxSpeed = kwargs.get("maxSpeed", 20.)
        arrowHeadAngle = kwargs.get("arrowHeadAngle", 35.)
        arrowHeadScale = kwargs.get("arrowHeadScale", 1.)
        arrowType = kwargs.get("arrowType", "barbed")
        palette = kwargs.get('palette', Viridis11)
        palette_reverse = kwargs.get('palette_reverse', False)
        
        if palette_reverse:
            palette.reverse()
        
        u = u[::ySkip, ::xSkip]
        v = v[::ySkip, ::xSkip]
            
        x = u.coord('longitude').points
        y = u.coord('latitude').points
        U = u.data
        V = v.data
        
        X, Y = np.meshgrid(x, y)
        speed = np.sqrt(U * U + V * V)
        
        # theta = np.arctan2(U, V)
        # r2d = 45.0 / math.atan(1.0)
        # theta = np.arctan2(U, V) + 45.*np.pi/180.
        # theta = -(np.arctan2(U, V) + np.pi / 2)
        # as per matplotlib.quiver code
        theta = np.arctan2(V, U)
        
        x0 = X.flatten()
        y0 = Y.flatten()
        length = speed.flatten() / maxSpeed
        angle = theta.flatten()
        x1 = x0 + length * np.cos(angle)
        y1 = y0 + length * np.sin(angle)

        # Colors
        cm = np.array(palette)
        #ix = ((length - length.min()) / (length.max() - length.min()) * (maxSpeed)).astype('int')
        ix = [int(i) for i in np.interp(length, (length.min(), length.max()), (0, len(cm)-1))]
        
        self.colors = cm[ix]
        #print(min(ix), max(ix), len(cm))
        
        dx = x1 - x0
        dy = y1 - y0

        rad = math.radians(arrowHeadAngle)  # ; //35 angle, can be adjusted
        # This is for Kite shaped arrows
        xR = x1 - arrowHeadScale * length  * np.cos(angle + rad)
        yR = y1 - arrowHeadScale * length * np.sin(angle + rad)

        xL = x1 - arrowHeadScale * length  * np.cos(angle - rad)
        yL = y1 - arrowHeadScale * length  * np.sin(angle - rad)
        
        if arrowType in ['kite', 'Kite', 'KITE']:
            self.xs = [[x1[i],xR[i],x0[i],xL[i],x1[i]] for i in range(len(x0))]
            self.ys = [[y1[i],yR[i],y0[i],yL[i],y1[i]] for i in range(len(y0))]
            
        if arrowType in ['barbed', 'Barbed', 'BARBED']:
            # for barbed arrows --|>
            xR1 = x1 - arrowHeadScale * length*0.5 * np.cos(angle + rad*0.5)
            yR1 = y1 - arrowHeadScale * length*0.5 * np.sin(angle + rad*0.5)

            xL1 = x1 - arrowHeadScale * length*0.5 * np.cos(angle - rad*0.5)
            yL1 = y1 - arrowHeadScale * length*0.5 * np.sin(angle - rad*0.5)
            
            self.xs = [[x1[i],xR[i],xR1[i],x0[i],xL1[i],xL[i],x1[i]] for i in range(len(x0))]
            self.ys = [[y1[i],yR[i],yR1[i],y0[i],yL1[i],yL[i],y1[i]] for i in range(len(y0))]


In [4]:
vec = vector(u850_ens_mean[0], v850_ens_mean[0], xSkip=5, ySkip=10)

In [85]:
def segment(x, y, U, V, density = 2, maxspeed=40):

    X, Y = np.meshgrid(x, y)
    speed = np.sqrt(U * U + V * V)
    theta = np.arctan(V / U)

    x0 = X[::density, ::density].flatten()
    y0 = Y[::density, ::density].flatten()
    length = speed[::density, ::density].flatten() / maxspeed
    angle = theta[::density, ::density].flatten()
    x1 = x0 + length * np.cos(angle)
    y1 = y0 + length * np.sin(angle)

    return x0, y0, x1, y1, length




In [86]:
u = u850_ens_mean[0,:5,:5]
v = v850_ens_mean[0,:5,:5]
x = u.coord('longitude').points
y = u.coord('latitude').points

In [5]:
vec = vector(u850_ens_mean[0], v850_ens_mean[0], xSkip=7, ySkip=7, 
             maxSpeed=5, arrowType='barbed', arrowHeadScale=0.2)

In [6]:
len(vec.colors)

806

In [8]:
arrow_source = ColumnDataSource(dict(xs=vec.xs, ys=vec.ys, colors=vec.colors))

plot = figure(plot_height=400, plot_width=1000, title='',
                           tools=["pan, reset, save, box_zoom, wheel_zoom, hover"],
                          x_axis_label='Longitude', y_axis_label='Latitude')
thickness_scale=10
plot.patches(xs="xs", ys="ys", fill_color="colors", line_color="colors", fill_alpha = 0.7, source=arrow_source)

show(plot)

GlyphRenderer(id='3914', ...)

In [110]:
xs=np.array([[[ [1, 1, 2, 2], [1.2, 1.6, 1.6], [1.8, 1.8, 1.6] ], [ [3, 3, 4] ]],
        [[ [1, 2, 2, 1], [1.3, 1.3, 1.7, 1.7] ]]])

In [114]:
xs[0]#.shape

[[[1, 1, 2, 2], [1.2, 1.6, 1.6], [1.8, 1.8, 1.6]], [[3, 3, 4]]]